# Two guys meet

In [1]:
using Sim, Printf

Since we implement the encounter as state machines, wee need some definitions. For simplicity we don't implement a state here but only events.

In [2]:
struct Guy
    name
end

abstract type Encounter end
struct Meet <: Encounter
    someone
end
struct Greet <: Encounter
    type
    from
end
struct Response <: Encounter
    type
    from
end

We have some convenience functions:

In [3]:
comm = ("Nice to meet you!", "How are you?", "Have a nice day!", "bye bye")
say(name, n) =  @printf("%5.2f s, %s: %s\n", now(sim), name, comm[n])

say (generic function with 1 method)

Now we implement the actions between the state machines as `step!`-function. In Julia we can implement it with different methods, which will be called according to the event. 

In [4]:
function step!(me::Guy, σ::Meet)
    event!(sim, SimFunction(step!, σ.someone, Greet(1, me)), after, 2*rand())
    say(me.name, 1)
end

function step!(me::Guy, σ::Greet)
    if σ.type < 3
        event!(sim, SimFunction(step!, σ.from, Response(σ.type, me)), after, 2*rand())
        say(me.name, σ.type)
    else
        say(me.name, 4)
    end
end

function step!(me::Guy, σ::Response)
    event!(sim, SimFunction(step!, σ.from, Greet(σ.type+1, me)), after, 2*rand())
    say(me.name, σ.type+1)
end

step! (generic function with 3 methods)

Now we need a simulation clock and two guys (state machines), define a starting event and run it.

In [5]:
sim = Clock()
foo = Guy("Foo")
bar = Guy("Bar")

event!(sim, SimFunction(step!, foo, Meet(bar)), at, 10*rand())
run!(sim, 20)

 9.95 s, Foo: Nice to meet you!
11.78 s, Bar: Nice to meet you!
12.05 s, Foo: How are you?
13.92 s, Bar: How are you?
14.37 s, Foo: Have a nice day!
14.99 s, Bar: bye bye
Finished: 6 events, simulation time: 20.0
